Using the good old MNIST digits, we'll explore hyper parameters. First up -- let's load and normalize the digits.

In [29]:
import numpy as np
import seaborn as sns
import math

import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D, Conv2D, BatchNormalization, ZeroPadding2D, Reshape
from keras.models import Model, Sequential
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit
import keras.wrappers.scikit_learn
import scipy.stats

sns.set()

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
train_images = np.expand_dims(x_train / np.max(x_train), -1)
test_images = np.expand_dims(x_test / np.max(x_test), -1)
train_labels = keras.utils.to_categorical(y_train, 10)
test_labels = keras.utils.to_categorical(y_test, 10)
train_images.shape, train_labels.shape

((60000, 28, 28, 1), (60000, 10))

This is the same build function we used with Grid Search, but just as a reminder -- the idea is to adapt Keras interface to Scikit learn interface.

Scikit learn has a great 'standardized' interface that we can use with keras. This requires a 'build function' -- which creates a model.

This build function has a set of parameters -- hyperparameters -- that we will vary in order to find a 'best' model.


In [5]:
input_shape = train_images[0].shape
num_classes = 10

def builder(activation='relu', depth=1, pool=3, 
            strides=2, filters=32, hidden=64, dropout=0.0,
            loss='categorical_crossentropy', optimzer='adam'):
    model = Sequential()
    # initial reshape to have consistent layering
    model.add(Reshape(input_shape, input_shape=input_shape))
    
    # convolutional stack
    for i in range(depth):
        model.add(Conv2D(filters, pool, activation=activation))
        model.add(ZeroPadding2D(pool//2))
        model.add(MaxPooling2D(pool, strides=strides))
        if dropout > 0:
            model.add(Dropout(dropout))
    
    # multilayer perceptron
    model.add(Flatten())
    for i in range(depth):
        model.add(Dense(hidden, activation=activation))
        if dropout > 0:
            model.add(Dropout(dropout))
    # final class activation
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss=loss, optimizer=optimzer, metrics=['accuracy'])
    return model

classifier = keras.wrappers.scikit_learn.KerasClassifier(builder)

In [5]:
classifier.fit(train_images, train_labels)

Epoch 1/1
60000/60000 [==============================] - 18s 303us/step - loss: 0.1622 - acc: 0.9527


Since we already have separated testing data, we'll use a custom cross validator to split at the index point where the training images end, and then just concatenate the training and testing data end to end. 

The `PredefinedSplit` let's us declare which samples are training `-1` and testing folds. We'll use only one testing fold, with a value of `0`, which is the 0 index.

In [1]:
all_images = np.concatenate((train_images, test_images), axis=0)
all_labels = np.concatenate((train_labels, test_labels), axis=0)
train_test_bitmap = np.concatenate(
    (np.full(len(train_images), -1), np.zeros(len(test_images))),
    axis=0)

train_test_data = PredefinedSplit(train_test_bitmap)


NameError: name 'np' is not defined

Now the actual randomized search. This is different than grid search in that each iteration, which is a kind of meta-epoch, takes a random sample for each parameter. This sampling is done on a probability distribution, so you need to think about what kind of numbers you need `int` or `float`, as well as the range of values that you think are valid.

And -- you need to pick a probability distribution, whether something flat, a uniform distribution like how dice or a quarter work, or something more sophisticated like a gaussian or an exponential that samples values with different probabilities.

There are no hard and fast rules to the best distribution to use, they are arrived at by experiment.

## Exponential
Here is a look at what the values are like in an exponential, first with the default scale of 1, and then with a scale of 0.1.

Scikit learn is taking a large number of random samples, so we'll simulate that by making a list of sampled values.

In [36]:
sample_indexes = [i for i in range(0, 100)]
exponential = scipy.stats.expon()
samples = [exponential.rvs() for x in sample_indexes]
sns.scatterplot(samples)

AttributeError: module 'seaborn' has no attribute 'scatterplot'

## Uniform
This is really two options -- uniform float and uniform integers, let's take a look at both. You'll see the numbers are evenly spread across the range.

In [37]:
sample_indexes = [i for i in range(0, 100)]
uniform = scipy.stats.uniform(1, 10)
samples = [uniform.rvs() for x in sample_indexes]
sns.scatterplot(samples)

AttributeError: module 'seaborn' has no attribute 'scatterplot'

In [39]:
sample_indexes = [i for i in range(0, 100)]
integers = scipy.stats.randint(32, 64)
samples = [integers.rvs() for x in sample_indexes]
sns.scatterplot(samples)

AttributeError: module 'seaborn' has no attribute 'scatterplot'

## Normal
Use this when you want numbers to hover around a midpoint, by default around zero. Now notice in this graph that this probably doesn't look like the bell curve you would expect -- that's because we have the sample index as the X axis. What you can see is the density is around zero -- that's the peak of the bell.

In [41]:
sample_indexes = [i for i in range(0, 100)]
normal = scipy.stats.norm(loc=0)
samples = [normal.rvs() for x in sample_indexes]
sns.scatterplot(samples)

AttributeError: module 'seaborn' has no attribute 'scatterplot'

Randomized search doesn't cover every possible permutation of parameters, it runs for `n_iter` iterations. Just a warning, this is going to take a **long while to run**. However, you can control the number of iterations to suit your time, patience, or compute budget.

The second thing to note is you can mix lists of parameters with distributions. The lists get a random pick from the options. The distributions get a random value generated. This is pretty cool, you can mix nominal parameters like activations with numerical parameters easily.


In [ ]:
param_grid = {
    'epochs': [4, 8],
    'activation': ['relu', 'elu'],
    'depth': [1, 2],
    'filters': [32, 64],
    'hidden': [64, 128],
    'dropout': [0.0, 0.25]
}
grid = RandomizedSearchCV(estimator=classifier, 
                    param_grid=param_grid, 
                    refit=False,
                    verbose=1,
                    n_iter=16,
                    cv=train_test_data)
grid_result = grid.fit(all_images, all_labels)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 1 folds for each of 64 candidates, totalling 64 fits
Epoch 1/4
60000/60000 [==============================] - 13s 209us/step - loss: 0.1551 - acc: 0.9535
Epoch 2/4
60000/60000 [==============================] - 12s 208us/step - loss: 0.0558 - acc: 0.9829
Epoch 3/4
60000/60000 [==============================] - 13s 213us/step - loss: 0.0391 - acc: 0.9881
Epoch 4/4
60000/60000 [==============================] - 5s 82us/step
Epoch 1/4
60000/60000 [==============================] - 13s 215us/step - loss: 0.1425 - acc: 0.9579
Epoch 2/4
47264/60000 [======================>.......] - ETA: 2s - loss: 0.0514 - acc: 0.9844

Machine learning is often a lot of human waiting, but it beats sitting and manually supervising experiments!